In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

%matplotlib inline

from tqdm import tqdm_notebook
from notifyme import notify

In [2]:
SHAPE = (137,236)
SHAPE_NEW = (128,128)

In [3]:
labels = pd.read_csv("./train.csv")
images = pd.read_parquet("./train_image_data_0.parquet")

In [4]:
labels.index = labels.image_id

In [5]:
labels = labels.loc[images.pop('image_id')]

In [6]:
def resize(img):
    return cv.resize(img.reshape(SHAPE),SHAPE_NEW).reshape(128*128)

In [7]:
images = np.apply_along_axis(resize,axis=1,arr=images.copy())

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split

In [17]:
graphime_root = labels.grapheme_root

In [18]:
graphime_root.nunique()

168

In [19]:
images = images.reshape(-1,128,128,1)

In [20]:
X,x,Y,y = train_test_split(images,graphime_root.values)

In [21]:
inputs = Input(shape = images.shape[1:])

model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu', input_shape=images.shape[1:])(inputs)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = Dropout(rate=0.3)(model)

model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = MaxPool2D(pool_size=(2, 2))(model)
model = Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
model = Dropout(rate=0.3)(model)

model = Flatten()(model)
model = Dense(1024, activation = "relu")(model)
model = Dropout(rate=0.3)(model)
dense = Dense(512, activation = "relu")(model)

dense = Dense(168, activation = 'softmax')(dense)

model = Model(inputs=inputs,outputs=[model])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

model.fit(X,Y,epochs=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 37657 samples
Epoch 1/5
28384/37657 [=====================>........] - ETA: 44s - loss: 16.3245 - acc: 0.0182